In [1]:
import numpy as np
import whisper
from whisper.decoding import DecodingOptions

tokenizer = whisper.tokenizer.get_tokenizer(multilingual=False)

# Build suppression lists once
SUPPRESS_TOKENS = [
        tokenizer.transcribe,
        tokenizer.translate,
        tokenizer.sot,
        tokenizer.sot_prev,
        tokenizer.sot_lm,
        tokenizer.timestamp_begin,
        tokenizer.no_timestamps,
    ]

special_ids = set(getattr(tokenizer, "special_tokens", {}).values())
SUPPRESS_TOKENS += list(special_ids)

# 2) Common junk tokens (constructed via encode, robust across vocab versions)
junk_pieces = [
    "\u2581", "▁",  # sentencepiece blanks (varies across forks)
    "\n", "\r", "\t",
    "  ", "   ",    # repeated spaces
]
junk_ids = set()
for t in junk_pieces:
    ids = tokenizer.encode(t, allowed_special=set())
    junk_ids.update(ids)


SUPPRESS_TOKENS += list(junk_ids)
SUPPRESS_TOKENS = list(set(SUPPRESS_TOKENS))


def apply_logit_filters(logits: np.ndarray):
    last = logits[:, -1, :]
    last[:, SUPPRESS_TOKENS] = -1e9
    logits[:, -1, :] = last
    return logits

In [3]:
import numpy as np
import whisper
import onnxruntime as ort
import librosa

tokenizer = whisper.tokenizer.get_tokenizer(multilingual=False)

enc = ort.InferenceSession("encoder.onnx")
dec = ort.InferenceSession("decoder.onnx")

# Use librosa instead of whisper.load_audio() to avoid ffmpeg dependency
audio_path = r"C:\Users\muthuvu\Downloads\reauthor-whisper-asr-android-runtime-master\sample.mp3"
audio, sr = librosa.load(audio_path, sr=16000)  # Load at 16kHz like whisper does

# Pad or trim to 30 seconds (whisper's expected length)
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio).unsqueeze(0).numpy().astype(np.float32)

audio_features = enc.run(None, {"mel": mel})[0]

# CRITICAL FIX: match export shape, then slice to empty
cache = np.zeros((8,1,1,384), dtype=np.float32)
print(cache.shape)

cache = cache[:, :, :0, :]   # now cache length = 0
print(cache.shape)
tokens = np.array([[tokenizer.sot, tokenizer.no_timestamps]], dtype=np.int64)

logits, cache = dec.run(None, {
    "tokens": tokens,
    "audio": audio_features,
    "cache": cache,
})

result = tokens[0].tolist()

for step in range(10):
    # Filter logits BEFORE argmax
    logits = apply_logit_filters(logits)

    next_token = np.argmax(logits[:, -1, :], axis=-1).reshape(1, 1).astype(np.int64)

    if next_token[0, 0] == tokenizer.eot:
        break

    result.append(int(next_token[0, 0]))

    logits, cache = dec.run(None, {
        "tokens": next_token,
        "audio": audio_features,
        "cache": cache,
    })

# Prefer skipping special tokens when printing
print(tokenizer.decode(result))

(8, 1, 1, 384)
(8, 1, 0, 384)
<|startoftranscript|><|notimestamps|> Once the,,,,,,,,
